<a href="https://colab.research.google.com/github/RS-Sowmya/Learning/blob/main/kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google .colab import files
upload = files.upload()

Saving kaggle_train.csv to kaggle_train.csv


In [3]:
df = pd.read_csv("kaggle_train.csv")

In [8]:
df.isnull().sum()

,0
id,0
PROJECTNUMBER,0
MW,10736
PROJECT_CITY,0
STATE,0
PROJECT_COUNTRY,0
CORE_MARKET,0
PROJECT_TYPE,2
SIZE_BUILDINGSIZE,30
NUMFLOORS,4449


In [14]:
df.sample(5)

,id,PROJECTNUMBER,MW,PROJECT_CITY,STATE,PROJECT_COUNTRY,CORE_MARKET,PROJECT_TYPE,SIZE_BUILDINGSIZE,NUMFLOORS,...,MasterItemNo,QtyShipped,UOM,ExtendedQuantity,PriceUOM,UnitPrice,ExtendedPrice,CONSTRUCTION_START_DATE,SUBSTANTIAL_COMPLETION_DATE,REVISED_ESTIMATE
2391,8566,A0-0028,NaN,Agra,Uttar Pradesh,India,Wellness,Ambulatory Care,4500.0,0.0,...,63778,20,PC,0.2,ML,118570.000,23714.00,1/23/2023 0:00,7/7/2023 12:00,1.652639e+08
5824,11464,A0-0023,NaN,Kolkata,West Bengal,India,Enterprise,Hospitality Hall,435000.0,11.0,...,59992,102,PC,1.02,ML,47261.000,48206.22,8/15/2022 0:00,1/21/2025 0:00,2.320456e+10
11959,14645,A0-0547,30.0,Tiruchirappalli,Tamil Nadu,India,Future Tech,Data Center,235000.0,7.0,...,CHILLER_TON,1182,TON,1182,TON,46709.900,55211101.80,3/1/2019 0:00,3/22/2027 0:00,2.270955e+10
7621,9173,A0-0202,NaN,Mangalore,Karnataka,India,Future Tech,Workspace,330000.0,10.0,...,65383,8,PC,8,PC,15030.000,120240.00,3/1/2022 0:00,7/31/2023 0:00,6.213316e+09
6758,6298,A0-0023,NaN,Kolkata,West Bengal,India,Enterprise,Hospitality Hall,435000.0,11.0,...,66754,50,BG,50,BG,1022.875,51143.75,8/15/2022 0:00,1/21/2025 0:00,2.320456e+10


In [48]:
df['ExtendedQuantity'] = pd.to_numeric(df['ExtendedQuantity'], errors='coerce')
df['QtyShipped'] = pd.to_numeric(df['QtyShipped'], errors='coerce')
from sklearn.impute import SimpleImputer
df_for_imp = df[['MW','SIZE_BUILDINGSIZE', 'NUMFLOORS', 'NUMROOMS','NUMBEDS', 'invoiceTotal', 'UnitPrice', 'ExtendedPrice','QtyShipped', 'ExtendedQuantity']]
def impute_numerical(df):
    df_imputed = df.copy()
    num_cols = df_imputed.select_dtypes(include=['int64', 'float64']).columns
    for col in num_cols:
        skewness = df_imputed[col].dropna().skew()
        if -0.5 <= skewness <= 0.5:
            imputer = SimpleImputer(strategy='mean')
        else:
            imputer = SimpleImputer(strategy='median')
        df_imputed[[col]] = imputer.fit_transform(df_imputed[[col]])
    df.update(df_imputed)
    return df
df[['MW','SIZE_BUILDINGSIZE', 'NUMFLOORS', 'NUMROOMS','NUMBEDS', 'invoiceTotal', 'UnitPrice', 'ExtendedPrice','QtyShipped', 'ExtendedQuantity']] = impute_numerical(df_for_imp)

In [49]:
df.isnull().sum()

,0
id,0
PROJECTNUMBER,0
MW,0
PROJECT_CITY,0
STATE,0
PROJECT_COUNTRY,0
CORE_MARKET,0
PROJECT_TYPE,2
SIZE_BUILDINGSIZE,0
NUMFLOORS,0
